In [2]:
import math

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [3]:
NUM_CLASSES = 10

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

print("x_train.shape = {}, y_train.shape = {}".format(x_train.shape, y_train.shape))
print("x_test.shape = {}, y_test.shape = {}".format(x_test.shape, y_test.shape))

x_train.shape = (60000, 28, 28, 1), y_train.shape = (60000, 10)
x_test.shape = (10000, 28, 28, 1), y_test.shape = (10000, 10)


In [5]:
inputs = Input(shape=(28, 28, 1), name='input')

x = Conv2D(24, kernel_size=(6, 6), strides=1)(inputs)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(48, kernel_size=(5, 5), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(64, kernel_size=(4, 4), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Flatten()(x)
x = Dense(200)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

predications = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=predications)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 23, 23, 24)        888       
                                                                 
 batch_normalization (BatchN  (None, 23, 23, 24)       72        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 23, 23, 24)        0         
                                                                 
 dropout (Dropout)           (None, 23, 23, 24)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 48)        28848     
                                                             

In [8]:
lr_decay = lambda epoch: 0.0001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

history = model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, 
                    validation_data=(x_test, y_test), callbacks=[decay_callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.0201.
Epoch 1/20
469/469 [==============================] - 170s 364ms/step - loss: 0.0929 - accuracy: 0.9710 - val_loss: 0.0458 - val_accuracy: 0.9848 - lr: 0.0201

Epoch 2: LearningRateScheduler setting learning rate to 0.014430626211475785.
Epoch 2/20
469/469 [==============================] - 161s 344ms/step - loss: 0.0573 - accuracy: 0.9824 - val_loss: 0.0330 - val_accuracy: 0.9883 - lr: 0.0144

Epoch 3: LearningRateScheduler setting learning rate to 0.01036834238065184.
Epoch 3/20
469/469 [==============================] - 161s 344ms/step - loss: 0.0388 - accuracy: 0.9881 - val_loss: 0.0224 - val_accuracy: 0.9928 - lr: 0.0104

Epoch 4: LearningRateScheduler setting learning rate to 0.007457588823428847.
Epoch 4/20
469/469 [==============================] - 178s 380ms/step - loss: 0.0325 - accuracy: 0.9899 - val_loss: 0.0235 - val_accuracy: 0.9928 - lr: 0.0075

Epoch 5: LearningRateScheduler setting learning rate to 0.0053

In [11]:
model.save('myhand_CNN_model.h5')

In [10]:
#converter = tf.lite.TFLiteConverter.from_keras_model_file(model)
#tflite_model = converter.convert()
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open('mnist.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\dudgm\AppData\Local\Temp\tmpj_9esero\assets


INFO:tensorflow:Assets written to: C:\Users\dudgm\AppData\Local\Temp\tmpj_9esero\assets


1147780

In [12]:
try:
    from google.colab import files
    files.download('mnist.tflite')
except:
    print("Skip downloading")

Skip downloading
